In [ ]:
# setup and imports

# the usual suspects
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm
import seaborn as sns

from ipywidgets import interact

# used for generating graphs
from graphviz import Digraph

# data mining algorithms
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn_extra.cluster import KMedoids

# config
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
# load the data
# note: this if randomly generated fake data, not real enrolment data
df = pd.read_csv('sample.csv')
df.drop('UNIQUEID', axis=1, inplace=True)

# Course Co-occurence Analysis

Learning Analytics, Visual Analytics @ UBC

Craig Thompson, CTLT

Academic programs often prescribe some number of official pathways. However, students may also choose to take combinations of courses other than those we intend.

We'd like to reveal those patterns.

![desire path](https://live.staticflickr.com/3203/2847766967_8e7ae25768_h.jpg)


[Desire path](https://flic.kr/p/5kDxUt) by [wetwebwork](https://www.flickr.com/photos/wetwebwork/][/url]), on Flickr [(CC BY 2.0)](https://creativecommons.org/licenses/by/2.0/)

# Data we have

- For every student who earns a degree, we have a record of all the courses they took and counted towards their degree requirements.

- We've limited the dataset to courses from a single department.

- Our dataset is two dimensional **binary indicator matrix**:
  - Across the horizontal axis: all the courses offered by the department
  - Down the vertical axis: IDs for each student who earned a degree
  - For each student/course pair we indicate whether the student took the course
  - This is fake data

In [ ]:
df.head(20)

# Data we don't have

- We don't have data for non-majors or anyone who did not complete their degree.

- We don't have performance data, so we don't know how well any student did in any particular course.

- We don't have any temporal information, so we don't know:
  - Which courses students took in sequence
  - Whether they took a pair of courses in back to back terms or with gaps
  - Which courses they took in concurrently

# About the analysis

- We are doing an exploratory analysis. This is not experimental.

- We will try to answer questions about *what* has happened, but we are unable to address *why*.

- We'd like to say "students are discovering their own pathways through our planned degrees". The reality is that students may be taking these groupings of courses because they:
  - Fit nicely in their timetable
  - Are offered by instructors they like
  - Have a reputation of being easy or fun courses
  - Have free or inexpensive textbooks
  - Are the only courses left at registration time

# Analysis via clustering

Common questions:
- What does an "average" student look like, in terms of the courses they study?
- If there were $N$ prototypical students, what would they look like?

Answer:
- We can formulate a *mathematically* average student, but there is no *pedagogically meaningful* average student.
- This sort of analysis is messy and hard to interpret.
- We'll do it anyway just to see!

In [ ]:
im = plt.imshow(df.head(100), cmap=plt.cm.binary)

In [ ]:
# most common courses
df.sum().sort_values(ascending=False).head(10)

In [ ]:
# how many courses does each student take?
df.sum(axis=1).value_counts().sort_index().plot(kind='bar');

In [ ]:
# helper functions for clustering students in course-space

def cluster(n, df):
    kmedoids = KMedoids(n_clusters=n, metric='manhattan', random_state=0).fit(df)
    nearest_medoid = kmedoids.predict(df)
    distances = kmedoids.transform(df)
    nearest_distance = distances[[np.arange(distances.shape[0])],nearest_medoid].T
    return (kmedoids, nearest_medoid, distances, nearest_distance)

def describe_clusters(kmedoids, nearest_medoid, distances, nearest_distance):
    plt.figure(figsize=(10, 10))

    for i in range(kmedoids.cluster_centers_.shape[0]):
        print("cluster", i+1, "centroid:", list(df.columns[kmedoids.cluster_centers_[i,:] == 1]))
        print("number of students in this cluster:", (nearest_medoid == i).sum())
        cluster_member_distances = nearest_distance[nearest_medoid == i]
        if cluster_member_distances.size > 0:
            print("minimum distance to centroid:", cluster_member_distances.min())
            print("maximum distance to centroid:", cluster_member_distances.max())
            print("mean distance to centroid:", cluster_member_distances.mean())
            print("median distance to centroid:", np.median(cluster_member_distances))
            print()
        plt.plot(sorted(cluster_member_distances))



In [ ]:
describe_clusters(*cluster(4, df))

# Lessons (re-)learned

- Course enrolment datasets are big, and hard to construct a clear mental picture of.
- We're working with (only) 50 students and 22 courses.
- Within academic programs, there aren't usually clear, strong, non-prescribed patterns at the level of whole-enrolment histories

So, let's try something different...

# History

What other domains work with similarly shaped data? Consumer purchases!

- Each individual shopper collects a bunch of items
- When a customer checks out, a sales invoice is generated listing all the items that were purchased together

From all the sales invoices, we may wish to look for patterns in consumer behaviour:
- Are there items that are **frequently** purchased together?
- Are some items good **predictors** of other items being purchased?

Why would someone care?
- If consumers buy hot dogs whenever they buy hotdog buns, then grocery stores can attempt to manipulate custormers into buying hotdogs by putting hotdog buns on sale. **Profit!**

# Content warning

The following slides contain math.

# Set theory

- a *set* is an unordered collection of distinct objects.

- For this analysis, each student's course enrolment history is being treated as a set. Sets are often written like this: $\{a,b,c\}$

- All the student enrolment histories are jointly represented as a collection of sets.
  - They are not a set-of-sets, because sets have distinct elements, and two students are able to have exactly the same course enrolment history.
  - So, this collection of sets is called a *multiset* or a *bag*, to denote that it may contain duplicate elements.



# Frequent itemsets

Given a multiset (such as a stack of grocery store receipts, or a table of student-course enrolments), how do we find the frequently occurring subsets (or itemsets)?

Example: Given $[\{a\},\{a,b\},\{a,c\},\{a,b,c,d\}]$

We can see that:
- $\{a\}$ occurs in all 4 sets
- $\{a,b\}$ and $\{a,c\}$ each occur in 2 sets


<math>
\begin{align}
&\mathrm{Apriori}(T,\epsilon)\\
&\quad L_1 \gets \{\textrm{large 1 item sets}\}\\
&\quad k \gets 2\\
&\quad \textbf{while}\ L_{k-1} \neq \emptyset\\
&\quad \quad C_k \gets \{c = a \cup \{b\} \mid a \in L_{k-1} \land b \notin a, \{s \subseteq c \mid |s| = k - 1 \} \subseteq L_{k-1} \}\\
&\quad \quad \textbf{for}\ \textrm{transactions}\ t \in T\\
&\quad \quad \quad D_t \gets \{c \in C_k \mid c \subseteq t \}\\
&\quad \quad \quad \textbf{for}\ \textrm{candidates}\ c \in D_t\\
&\quad \quad \quad \quad count[c] \gets count[c] + 1\\
&\quad \quad L_k \gets \{c \in C_k \mid count[c] \geq \epsilon \}\\
&\quad \quad k \gets k + 1\\
&\quad \textbf{return} \bigcup_k L_k
\end{align}
</math>

Rakesh Agrawal and Ramakrishnan Srikant. 1994. Fast Algorithms for Mining Association Rules in Large Databases. In Proceedings of the 20th International Conference on Very Large Data Bases (VLDB ’94). Morgan Kaufmann Publishers Inc., San Francisco, CA, USA, 487–499. ( > 26k citations!)

Let $X$ be an itemset and $T$ be the set of transactions/records in the database


<math>
\begin{align}
&\textrm{Support}(X) = \frac{\mid \{t \in T \mid X \subseteq t \}\mid }{\mid T \mid}
\end{align}
</math>


*Support* indicates how frequently a given itemset appears in the transactions of the database.
- A support of 1 indicates the itemset appears in every transaction.
- A support of 0.5 indicates the itemset appears in half of the transactions.

In [ ]:
course_frequency = apriori(df, min_support=np.nextafter(0,1), max_len=1, use_colnames=True)
course_frequency['course'] = course_frequency['itemsets'].apply(lambda x: set(x).pop())
course_frequency['course_number'] = course_frequency['course'].apply(lambda x: x[4:])
course_frequency[['support', 'course']].sort_values(by='support',ascending=False)

In [ ]:
cf = course_frequency[['support', 'course']].set_index('course').sort_values(by='support',ascending=False)

def f(limit):
    cf.head(limit).plot(kind='bar')

i = interact(f, limit=(1,20))

In [ ]:
frequent_itemsets = apriori(df, min_support=np.nextafter(0,1), max_len=2, use_colnames=True)
frequent_itemsets.sort_values(by='support',ascending=False)

# Association rules

\begin{equation*}
X \Rightarrow Y, \textrm{where}\ X,Y \subseteq I
\end{equation*}

X is called the *antecedent* and Y is the *consequent*.

$I$ is the set of all items (e.g. courses).

example: $\textrm{Math110} \Rightarrow \textrm{Math210}$ would be read as "if Math110, then Math210".

Now we have a notation for a relationship between two itemsets (in this case, the two itemsets each contain a single item), but we need to describe the *qualities* of that relationship...

Rakesh Agrawal, Tomasz Imieliński, and Arun Swami. 1993. Mining association rules between sets of items in large databases. In Proceedings of the 1993 ACM SIGMOD international conference on Management of data (SIGMOD ’93). Association for Computing Machinery, New York, NY, USA, 207–216. DOI:https://doi.org/10.1145/170035.170072 (> 22k citations!)

# Metrics for quantifying association rules: Support

- *Antecedent Support*: indicates how frequently the antecedent item set appears in the database.

$$\textrm{Antecedent Support}(X \Rightarrow Y) = \frac{\mid \{t \in T \mid X \subseteq t \}\mid }{\mid T \mid}$$

- *Consequent Support*: indicates how frequently the consequent item set appears in the database.

$$\textrm{Consequent Support}(X \Rightarrow Y) = \frac{\mid \{t \in T \mid Y \subseteq t \}\mid }{\mid T \mid}$$

- *(Rule) Support*: indicates how frequently the all them items of the antecedent and consequent jointly appear in the database.

$$\textrm{Support}(X \Rightarrow Y) = \frac{\mid \{t \in T \mid X \cup Y \subseteq t \}\mid }{\mid T \mid}$$

## Metrics for quantifying association rules: Confidence

$$\textrm{Confidence}(X \Rightarrow Y) = \frac{ \textrm{Support}(X \Rightarrow Y) }{ \textrm{Support}(X) }$$

*Confidence*: the ratio of rule support to antecedent support. 
  - Or, given that the antecedent has been observed, how likely are we to also observe the consequent?

If a rule has high confidence, and the antecedent is observed, then we can be fairly confident that the consequent will be observed as well.

## Metrics for quantifying association rules: Lift

$$\textrm{Lift}(X \Rightarrow Y) = \frac{ \textrm{Confidence}(X \Rightarrow Y) }{ \textrm{Support}(Y) }$$

*Lift*: ratio of confidence to consequent support. Lift is a measure of how much more often the antecedent and the consequent occur together than would be expected if they were statistically independent. When the antecedent of a rule with high lift is observed, we can be more confident that the consequent will also be observed.

Confidence and lift are both descriptors of the "power" of a rule.

In [ ]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=np.nextafter(0,1))

rules['antecedent_course'] = rules['antecedents'].apply(lambda x: set(x).pop())
rules['consequent_course'] = rules['consequents'].apply(lambda x: set(x).pop())

rules['antecedent_course_number'] = rules['antecedent_course'].apply(lambda x: int(x[4:]))
rules['consequent_course_number'] = rules['consequent_course'].apply(lambda x: int(x[4:]))

rules['antecedent_year_level'] = rules['antecedent_course_number'].apply(lambda x: x//100 )
rules['consequent_year_level'] = rules['consequent_course_number'].apply(lambda x: x//100)

In [ ]:
rules

In [ ]:
pairwise_rules = rules[(rules['antecedent_year_level']==3) & (rules['consequent_year_level']==3)]
pairwise_support = pairwise_rules.pivot(index='antecedent_course',columns='consequent_course',values='support').fillna(0)
ax = sns.heatmap(pairwise_support, xticklabels=True, yticklabels=True, cmap='BuPu')

In [ ]:
pairwise_rules = rules[(rules['antecedent_year_level']==3) & (rules['consequent_year_level']==3)]
pairwise_confidence = pairwise_rules.pivot(index='antecedent_course',columns='consequent_course',values='confidence').fillna(0)
ax = sns.heatmap(pairwise_confidence, xticklabels=True, yticklabels=True, cmap='BuPu')

In [ ]:
pairwise_rules = rules[(rules['antecedent_year_level']==3) & (rules['consequent_year_level']==3)]
pairwise_lift = pairwise_rules.pivot(index='antecedent_course',columns='consequent_course',values='lift').fillna(0.1)
#pairwise_lift = pairwise_lift.applymap(lambda x: x if x >=1 else 0.01)
ax = sns.heatmap(pairwise_lift, xticklabels=True, yticklabels=True, cmap='BuPu', norm=LogNorm())

In [ ]:
# exploring 'significant' rules
sig_rules = rules[
      (rules['support'] > 0.01)
    #& (rules['antecedent support'] > 0.01)
    #& (rules['consequent support'] > 0.01)
    & (rules['antecedent_year_level'] <= rules['consequent_year_level'])
    & (rules['confidence'] > 0.5)
    & (rules['lift'] > 1.5)
                 ].sort_values(by='lift',ascending=False)
sig_rules

In [ ]:
def plot_rules(sig_rules):
    antecedents = sig_rules[['antecedent_course','antecedent_course_number']]
    antecedents.columns = ['course','course_number']

    consequents = sig_rules[['consequent_course','consequent_course_number']]
    consequents.columns = ['course','course_number']

    figure_courses = pd.concat([antecedents, consequents]).drop_duplicates()

    dot = Digraph()
    for course in figure_courses.itertuples():
        dot.node(str(course.course_number),course.course)

    for association in sig_rules.itertuples():
        dot.edge(str(association.antecedent_course_number), str(association.consequent_course_number), label=f"{association.lift:.2f}")

    dot.graph_attr['overlap'] = 'False'
    dot.engine = 'neato'
    return dot

In [ ]:
dot = plot_rules(sig_rules)
dot

# What next?

- This analysis was (nearly) totally naive to the official structure of the courses/programs offered. That means we 'discovered' several results that are obvious and uninteresting to anyone with domain expertise.
  - For example, we may have 'discovered' that if you take a senior course, then you are very likely to also take the prerequisites.
  - We should filter the result set to remove any association rules that reflect formal prerequisite relationships.

- We've looked at *what* courses students take, but we have not investigated ordering, or the effects of ordering. An analysis of sequencing would be useful for planning connections across courses.

# Thank You

Questions?

Contact: craig.thompson@ubc.ca